In [1]:
import pandas as pd

In [2]:
df = pd.read_json(path_or_buf='gg2013.json')
# df = pd.read_json(path_or_buf='gg2015.json')
df.head(3)

,id,text,timestamp_ms,user
0,290620657987887104,JLo's dress! #eredcarpet #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}"
1,290620657887219713,What's making Sofia Vergara's boobs stay like ...,2013-01-14 00:45:38,"{'screen_name': 'theAmberShow', 'id': 14648726}"
2,290620657828524032,RT @FabSugar: Kerry Washington is EVERYTHING. ...,2013-01-14 00:45:38,"{'screen_name': 'SweetyPW', 'id': 35498686}"


In [3]:
data = df['text']
data[:5]

0               JLo's dress! #eredcarpet #GoldenGlobes
1    What's making Sofia Vergara's boobs stay like ...
2    RT @FabSugar: Kerry Washington is EVERYTHING. ...
3       Anne Hathaway has got me living. #GoldenGlobes
4    Jennifer Lopez's lace dress? Thoughts? #Golden...
Name: text, dtype: object

In [4]:
len(data)

174643

In [5]:
# sample data if necessary
sample_size = 200000
if len(df) > sample_size:
    data = data.sample(n=sample_size)
len(data)

174643

In [6]:
import re

In [7]:
retweets_freq_dict = {}

def remove_retweet_prefix(line):
    # find 'RT @abc: ' where abc's length is arbitrary
    pattern = re.compile(r'\bRT @([\w\'/]*)\b: ') 
    match = re.search(pattern, line)
    if match:
        # store corresponding retweet without 'RT @' prefix
        string = match.group()[4:]
        if string in retweets_freq_dict:
            retweets_freq_dict[string] += 1
        else:
            retweets_freq_dict[string] = 1
    return re.sub(pattern, ' ', line)

In [8]:
hashtag_freq_dict = {}

def remove_hashtag(line):
    pattern = re.compile(r'#([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in hashtag_freq_dict:
                hashtag_freq_dict[match] += 1
            else:
                hashtag_freq_dict[match] = 1
    return re.sub(pattern, ' ', line)

In [9]:
at_freq_dict = {}

def remove_at(line):
    pattern = re.compile(r'@([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in at_freq_dict:
                at_freq_dict[match] += 1
            else:
                at_freq_dict[match] = 1
    return re.sub(pattern, ' ', line)

In [10]:
def remove_url(line):
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\b')
    return re.sub(pattern, ' ', line)

In [11]:
def cleanse(line):
    # replace everything to ' ' except whitespace, alphanumeric character, apostrophe, hashtag, @
    return re.sub(r'[^\w\s\'#@]', ' ', line)

In [12]:
def remove_apostrophe(text):
    pattern = re.compile(r'\'s\b')
    return re.sub(pattern, ' ', text)

In [13]:
import spacy
# python3 -m spacy download en
nlp = spacy.load('en')

# reference for token attributes
# https://spacy.io/api/token#attributes

In [14]:
def identify_entities(text):
    entities = list(nlp(text).ents)
    tags = {}
    for entity in entities:
        if entity not in tags:
#             tags[' '.join(t.orth_ for t in entity).strip()]=[entity.label_]
#             print(entity.text)
#             print(' '.join(t.orth_ for t in entity).strip())
            tags[entity.text]=[entity.label_]
    return tags

# performance may be compared with nltk.tag.stanford.StanfordTagger if we have time
# http://www.nltk.org/api/nltk.tag.html#module-nltk.tag.stanford

# test
text = cleanse(remove_retweet_prefix(data[53365]))
tags = identify_entities(text)
print(text)
print(tags)
print(list(tags.keys())[0])

OMG   Adele  prepare to be the recipient of a very angry song from Taylor Swift #GoldenGlobes 
{'Adele  ': ['LANGUAGE'], 'Taylor Swift': ['PERSON']}
Adele  


In [15]:
%%time
# generate cleansed_data beforehand
cleansed_data = []
for tweet in data:
    # remove_retweet_prefix 
    line = remove_retweet_prefix(tweet)
    # remove hashtag
    line = remove_hashtag(line)
    # remove @...
    line = remove_at(line)
    # remove url
    line = remove_url(line)
    # remove punctuations except apostrophe
    line = cleanse(line)
    cleansed_data.append(line)

CPU times: user 3.9 s, sys: 98.6 ms, total: 3.99 s
Wall time: 3.42 s


In [16]:
cleansed_data[:5]

["JLo's dress     ",
 "What's making Sofia Vergara's boobs stay like that  Magic  Witchcraft   ",
 ' Kerry Washington is EVERYTHING  Dying over her Miu Miu gown       ',
 'Anne Hathaway has got me living   ',
 "Jennifer Lopez's lace dress  Thoughts   "]

In [17]:
def find_host(verbose=False):
    pattern = re.compile(r'\bhost')
    entity_freq_dict = {}
    
    num = 0
    max_entity_len = 0
    max_entity = None
    for line in cleansed_data:
        match = re.search(pattern, line.lower())
        if match:
            tags = identify_entities(line)

            if verbose:
                # print the first 10 occurrences
                if num < 10:
                    print(tweet)
                    print(line)
                    print(tags)
                    print()

            for entity in tags.keys():
                entity = entity.strip()
                # identify the entity with maximum length
                entity_len = len(entity)
                if entity_len > max_entity_len:
                    max_entity_len = entity_len
                    max_entity = entity
                
                if entity_len > 1:
                    if entity not in entity_freq_dict:
                        entity_freq_dict[entity] = 1 # tried adding more weights to 'PERSON' tags but results are not good
                    else:
                        entity_freq_dict[entity] += 1
            num += 1           
    print('num of matches:', num)
    print('max_entity_len:', max_entity_len)
    print('max_entity:', max_entity)
    return entity_freq_dict

In [18]:
%%time

# note that for 2015 tweets, it takes more than 4min to run
# processing time may be long for large data
# consider picking the longest 150000 tweets

entity_freq_dict = find_host(verbose=True)

Says @BenAffleck: "I also didn't get the Acting Nomination...no one's saying I got snubbed there!" #pressroom #GoldenGlobes
Looking forward to watching Tina Fey and Amy Poehler host the  
{'Tina Fey': ['EVENT'], 'Amy Poehler': ['PERSON']}

Says @BenAffleck: "I also didn't get the Acting Nomination...no one's saying I got snubbed there!" #pressroom #GoldenGlobes
 It's our hosts Tina Fey and Amy Poehler       
{'Tina Fey': ['PERSON'], 'Amy Poehler       ': ['PERSON']}

Says @BenAffleck: "I also didn't get the Acting Nomination...no one's saying I got snubbed there!" #pressroom #GoldenGlobes
 Tonight's dual hosting duties represent the culmination of a decade of Amy and Tina partnerships     
{'Tonight': ['TIME'], 'a decade': ['DATE'], 'Amy': ['PERSON'], 'Tina': ['GPE']}

Says @BenAffleck: "I also didn't get the Acting Nomination...no one's saying I got snubbed there!" #pressroom #GoldenGlobes
 My green suede tuxedo pinching a bit here at the Velvet Rope Awards honoring best in crowd cont

In [19]:
# top retweets (each as an entity)
sorted(retweets_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:50]

[('goldenglobes: ', 3459),
 ('eonline: ', 2891),
 ('PerezHilton: ', 2301),
 ('TheEllenShow: ', 1696),
 ('EmWatson: ', 1147),
 ('VanityFair: ', 820),
 ('nbcsnl: ', 743),
 ('CNNshowbiz: ', 699),
 ('CiudadBizarra: ', 510),
 ('BuzzFeed: ', 491),
 ('EW: ', 485),
 ('nbc: ', 465),
 ('vulture: ', 446),
 ('piersmorgan: ', 441),
 ('MARLONLWAYANS: ', 384),
 ('HuffingtonPost: ', 369),
 ('buckhollywood: ', 353),
 ('MarilynMonroeES: ', 334),
 ('TVGuide: ', 319),
 ('THR: ', 315),
 ('DavidSpade: ', 312),
 ('MTVNews: ', 307),
 ('PimpBillClinton: ', 281),
 ('cinema21: ', 281),
 ('washingtonpost: ', 275),
 ('ninagarcia: ', 258),
 ('Cosmopolitan: ', 239),
 ('peopleenespanol: ', 219),
 ('RichardCrouse: ', 217),
 ('peoplemag: ', 213),
 ('prodigalsam: ', 212),
 ('kumailn: ', 211),
 ('DannyZuker: ', 207),
 ('jianghomeshi: ', 201),
 ('HuffPostWomen: ', 200),
 ('DamienFahey: ', 200),
 ('girlsHBO: ', 198),
 ('MHarrisPerry: ', 194),
 ('DougBenson: ', 192),
 ('rogergzz: ', 191),
 ('InStyle: ', 191),
 ('EonlineLati

In [20]:
# top hashtags
sorted(hashtag_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:50]

[('GoldenGlobes', 110135),
 ('goldenglobes', 33113),
 ('Argo', 842),
 ('GetGlue', 769),
 ('Homeland', 696),
 ('redcarpet', 675),
 ('GoldenGlobe', 571),
 ('Goldenglobes', 560),
 ('JodieFoster', 486),
 ('Girls', 451),
 ('killingit', 447),
 ('GOLDENGLOBES', 420),
 ('AlfombraRojaE', 385),
 ('Skyfall', 360),
 ('GlobosdeOro', 352),
 ('RedCarpet', 348),
 ('Lincoln', 348),
 ('LesMis', 346),
 ('GIRLS', 332),
 ('Adele', 289),
 ('LesMiserables', 282),
 ('JenniferLawrence', 258),
 ('GG2013', 254),
 ('homeland', 251),
 ('GoldenGlobes2013', 249),
 ('TinaFey', 226),
 ('ERedCarpet', 224),
 ('AmyPoehler', 209),
 ('eredcarpet', 207),
 ('jodiefoster', 204),
 ('DowntonAbbey', 193),
 ('AnneHathaway', 191),
 ('girls', 188),
 ('DjangoUnchained', 184),
 ('Oscars', 176),
 ('LoveThoseLadies', 163),
 ('Django', 158),
 ('BenAffleck', 154),
 ('Globes', 149),
 ('GameChange', 146),
 ('LOVEHER', 146),
 ('goldenGlobes', 143),
 ('fb', 143),
 ('BillClinton', 142),
 ('fashion', 138),
 ('Oscar', 138),
 ('skyfall', 137),
 

In [21]:
# top @
sorted(at_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:50]

[('goldenglobes', 3150),
 ('OfficialAdele', 1122),
 ('lenadunham', 980),
 ('GoldenGlobes', 863),
 ('BenAffleck', 526),
 ('RealHughJackman', 386),
 ('PerezHilton', 361),
 ('SHO_Homeland', 357),
 ('girlsHBO', 352),
 ('SofiaVergara', 303),
 ('eonline', 290),
 ('taylorswift13', 270),
 ('TNTLA', 255),
 ('JLo', 235),
 ('PaulEpworth', 223),
 ('GirlsHBO', 182),
 ('msleamichele', 167),
 ('SelenaGomez', 164),
 ('VanessuHudgens', 163),
 ('LenaDunham', 156),
 ('steph_hart', 156),
 ('RyanGosling', 137),
 ('kerrywashington', 125),
 ('PixarBrave', 120),
 ('LesMiserables', 113),
 ('nbc', 112),
 ('Chanel', 112),
 ('NathanFillion', 105),
 ('KevalBaxi', 97),
 ('Burberry', 86),
 ('Lewis_Damian', 86),
 ('jessicaalba', 84),
 ('CHANEL', 83),
 ('tomandlorenzo', 78),
 ('nbcsnl', 77),
 ('iamdoncheadle', 76),
 ('piersmorgan', 69),
 ('CiudadBizarra', 67),
 ('CNNshowbiz', 64),
 ('BuzzFeed', 64),
 ('LeoDiCaprio', 62),
 ('EvaLongoria', 61),
 ('azizansari', 60),
 ('gabrielledoug', 60),
 ('Sarah_Hyland', 60),
 ("lenad

In [22]:
top_100 = sorted(entity_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:100]
top_100

[('Will Ferrell', 700),
 ('Amy Poehler', 604),
 ('Amy', 529),
 ('Kristen Wiig', 422),
 ('Tina', 418),
 ('next year', 377),
 ('Tina Fey', 352),
 ('Oscars next year', 258),
 ('2014', 165),
 ('Golden Globes', 99),
 ('Quentin Tarantino', 95),
 ('Best  Hosts', 89),
 ('Scratch', 62),
 ('the Golden Globes', 58),
 ('Opening Monologue', 54),
 ('Paul Rudd', 53),
 ('two', 50),
 ('Golden Globes Hosts Tina', 50),
 ('Amy Poelher', 48),
 ('SNL', 47),
 ('Thoroughly', 46),
 ('tonight', 43),
 ('Great', 42),
 ('Can Tina', 42),
 ("next year's", 41),
 ('RT', 37),
 ('this Golden Globes  Tina', 37),
 ('Iran', 30),
 ('Hollywood', 29),
 ('Please', 29),
 ('Academy', 28),
 ('every year', 25),
 ('Kristen Wiig and', 25),
 ('Canadians', 25),
 ('Canada', 25),
 ('Amy Poehler on Hosting Golden Globes', 23),
 ('Poehler', 22),
 ('this year', 21),
 ('Host', 21),
 ('Amy Pohler', 20),
 ('first', 19),
 ('Can Tina Fey', 18),
 ('Love', 17),
 ('2013', 16),
 ('So', 16),
 ("Amy Poehler's", 15),
 ('Oscars', 15),
 ('Run the World'

In [23]:
# pip3 install python-Levenshtein for 4-10x speedup
from fuzzywuzzy import fuzz

# test
print(fuzz.ratio('Tina Fey', 'Tina'))
print(fuzz.ratio('Amy Poehler', 'Amy'))
print(fuzz.ratio('Golden Globes', 'golden globes'))

67
43
85


In [24]:
# remove golden globes from names
import pprint
names = [pair[0] for pair in top_100]
golden_globes = [name for name in names if fuzz.ratio(name.lower(), 'golden globes') > 60]
pprint.pprint(golden_globes)

['Golden Globes',
 'the Golden Globes',
 'Golden Globes Hosts Tina',
 'this Golden Globes  Tina',
 'The Golden Globes']


In [25]:
# we also consider dropping all lower cases examples or examples that contain digit(s), which are not names
def filter_names(pair_list):
    filtered_results = []
    for pair in pair_list:
        string = ''.join(pair[0].split())
        if not all(char.islower() for char in string) and not any(char.isdigit() for char in string):
            filtered_results.append(pair)
        else:
            if pair[0] in entity_freq_dict:
                del entity_freq_dict[pair[0]]
    return filtered_results

In [26]:
for name in golden_globes:
    if name in entity_freq_dict:
        del entity_freq_dict[name]
top_results = sorted(entity_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:50]
top_results = filter_names(top_results)
top_results

[('Will Ferrell', 700),
 ('Amy Poehler', 604),
 ('Amy', 529),
 ('Kristen Wiig', 422),
 ('Tina', 418),
 ('Tina Fey', 352),
 ('Oscars next year', 258),
 ('Quentin Tarantino', 95),
 ('Best  Hosts', 89),
 ('Scratch', 62),
 ('Opening Monologue', 54),
 ('Paul Rudd', 53),
 ('Amy Poelher', 48),
 ('SNL', 47),
 ('Thoroughly', 46),
 ('Great', 42),
 ('Can Tina', 42),
 ("next year's", 41),
 ('RT', 37),
 ('Iran', 30),
 ('Hollywood', 29),
 ('Please', 29),
 ('Academy', 28),
 ('Kristen Wiig and', 25),
 ('Canadians', 25),
 ('Canada', 25),
 ('Amy Poehler on Hosting Golden Globes', 23),
 ('Poehler', 22),
 ('Host', 21),
 ('Amy Pohler', 20),
 ('Can Tina Fey', 18),
 ('Love', 17),
 ('So', 16),
 ("Amy Poehler's", 15),
 ('Oscars', 15),
 ('Run the World', 15),
 ('Wiig', 15),
 ('Quentin Tarantin', 15),
 ('Ricky Gervais', 13),
 ('Can', 13),
 ('Ferrell', 13)]

In [27]:
names = [pair[0] for pair in top_results]
names_clusters = []

for name in names:
    # each name starts as a cluster
    cluster = [name]
    names_to_reduce = names[:]
    names_to_reduce.remove(name)
    
    # one vs. all comparisons
    for i in names_to_reduce:
        ratio = fuzz.ratio(name.lower(), i.lower())
        # if similarity is larger than 75 or one name is contained in the other name
        if ratio > 75 or re.search(name, i, flags=re.IGNORECASE) or re.search(i, name, flags=re.IGNORECASE):
            cluster.append(i)
    
    # if multiple names are identified in one cluster
    if len(cluster) > 1:
        names_clusters.append(cluster)
    
    print(cluster)

# find names clusters that should merge
# ['Amy Poehler', 'Amy', 'Amy Poelher']
# ['Tina', 'Tina Fey']

# sort clusters
names_clusters.sort()
# sort within each cluster
names_clusters = ['|'.join(sorted(cluster)) for cluster in names_clusters]
# remove overlaps
names_clusters_reduced = [line.split('|') for line in list(set(names_clusters))]
# sort by length from shortest to longest (merge from the shortest)
names_clusters_reduced.sort(key=len)
print('\nnames clusters to merge:')
pprint.pprint(names_clusters_reduced)

['Will Ferrell', 'Ferrell']
['Amy Poehler', 'Amy', 'Amy Poelher', 'Amy Poehler on Hosting Golden Globes', 'Poehler', 'Amy Pohler', "Amy Poehler's"]
['Amy', 'Amy Poehler', 'Amy Poelher', 'Amy Poehler on Hosting Golden Globes', 'Amy Pohler', "Amy Poehler's"]
['Kristen Wiig', 'Kristen Wiig and', 'Wiig']
['Tina', 'Tina Fey', 'Can Tina', 'Can Tina Fey']
['Tina Fey', 'Tina', 'Can Tina Fey']
['Oscars next year', 'Oscars']
['Quentin Tarantino', 'Quentin Tarantin']
['Best  Hosts', 'Host']
['Scratch']
['Opening Monologue']
['Paul Rudd']
['Amy Poelher', 'Amy Poehler', 'Amy', 'Amy Pohler', "Amy Poehler's"]
['SNL']
['Thoroughly']
['Great']
['Can Tina', 'Tina', 'Can Tina Fey', 'Can']
["next year's"]
['RT']
['Iran']
['Hollywood']
['Please']
['Academy']
['Kristen Wiig and', 'Kristen Wiig', 'Wiig']
['Canadians', 'Canada', 'Can']
['Canada', 'Canadians', 'Can']
['Amy Poehler on Hosting Golden Globes', 'Amy Poehler', 'Amy', 'Poehler', 'Host']
['Poehler', 'Amy Poehler', 'Amy Poehler on Hosting Golden Globe

In [28]:
# weighted frequency of an entity is defined by its frequency multiplied by its string length
def weighted_freq(element):
    return entity_freq_dict[element] * len(element)

e = entity_freq_dict.copy()
for cluster in names_clusters_reduced:
    # select the entity name with highest weighted frequency
    selected_entity_name = max(cluster, key=weighted_freq)
    cluster.remove(selected_entity_name)
    # for names to be merged to the selected entity name
    for name in cluster:
        # if not deleted in previous cases, cumulate frequencies to the selected entity
        if name in e and selected_entity_name in e:
            e[selected_entity_name] += e[name]
            del e[name]

In [29]:
top_10 = sorted(e.items(), key=lambda pair: pair[1], reverse=True)[:10]
top_10

[('Amy Poehler', 1238),
 ('Tina Fey', 830),
 ('Will Ferrell', 713),
 ('Kristen Wiig', 462),
 ('Oscars next year', 273),
 ('Best  Hosts', 133),
 ('Quentin Tarantino', 110),
 ('Canadians', 63),
 ('Scratch', 62),
 ('Opening Monologue', 54)]

In [30]:
import json

# top 2 inferences for hosts
best_host_prediction = [name[0] for name in top_10][:2]
json.dumps({'Host': best_host_prediction})

'{"Host": ["Amy Poehler", "Tina Fey"]}'